# Scenario 02: AG-UI Protocol Interface

**Estimated Time**: 45 minutes

## Learning Objectives
- Understand the AG-UI streaming protocol
- Build an AG-UI compatible server with FastAPI
- Handle lifecycle and message events
- Integrate agents with frontend interfaces

## Prerequisites
- Completed Scenario 01 (Simple Agent + MCP)
- Basic understanding of Server-Sent Events (SSE)

## Part 1: Understanding AG-UI Protocol

### What is AG-UI?

AG-UI (Agent-User Interface) is a protocol for building streaming chat interfaces.
It defines how agents communicate with user interfaces in real-time.

### Key Concepts

1. **Server-Sent Events (SSE)**: One-way streaming from server to client
2. **Event Types**: Structured events for different message phases
3. **Lifecycle Events**: RUN_STARTED, RUN_FINISHED, RUN_ERROR
4. **Message Events**: TEXT_MESSAGE_START, TEXT_MESSAGE_CONTENT, TEXT_MESSAGE_END
5. **Tool Events**: TOOL_CALL_START, TOOL_CALL_ARGS, TOOL_CALL_END

### Event Flow

```
RUN_STARTED
    ↓
TEXT_MESSAGE_START
    ↓
TEXT_MESSAGE_CONTENT (repeated)
    ↓
TEXT_MESSAGE_END
    ↓
RUN_FINISHED
```

## Part 2: Setting Up the Environment

In [1]:
# Verify imports
import sys
import asyncio
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

print(f"✅ Project root: {project_root}")

# Ensure we can import from src
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import AG-UI components
from src.agents import AGUIServer, AGUIEventEmitter, EventType, create_agui_server
from src.agents.agui_server import (
    RunAgentInput,
    Message,
    BaseEvent,
    RunStartedEvent,
    TextMessageContentEvent,
)
from src.common.telemetry import setup_telemetry, get_tracer

# Setup telemetry
setup_telemetry()
tracer = get_tracer(__name__)

print("✅ AG-UI components imported successfully!")
print(f"\nAvailable event types: {[e.value for e in EventType]}")

✅ Project root: C:\Users\jonasrotter\OneDrive - Microsoft\Desktop\Jonas Privat\MyCodingProjects\agents-workshop


azure-monitor-opentelemetry-exporter not installed, falling back to console export


✅ AG-UI components imported successfully!

Available event types: ['RUN_STARTED', 'RUN_FINISHED', 'RUN_ERROR', 'TEXT_MESSAGE_START', 'TEXT_MESSAGE_CONTENT', 'TEXT_MESSAGE_END', 'TOOL_CALL_START', 'TOOL_CALL_ARGS', 'TOOL_CALL_END', 'STATE_SNAPSHOT', 'STATE_DELTA', 'RAW']


## Part 3: Event Emitter Deep Dive

The `AGUIEventEmitter` class handles creating and formatting AG-UI events.
Let's explore how it works.

In [2]:
# Create an event emitter
emitter = AGUIEventEmitter(
    thread_id="thread-demo-123",
    run_id="run-demo-456"
)

# Emit lifecycle events
print("=== Run Lifecycle Events ===")
print(emitter.emit_run_started())
print(emitter.emit_run_finished())

=== Run Lifecycle Events ===
event: message
data: {"type":"RUN_STARTED","timestamp":1767345861823,"raw_event":null,"thread_id":"thread-demo-123","run_id":"run-demo-456"}


event: message
data: {"type":"RUN_FINISHED","timestamp":1767345861823,"raw_event":null,"thread_id":"thread-demo-123","run_id":"run-demo-456"}




In [ ]:
# Emit text message events
print("=== Text Message Events ===")
print(emitter.emit_text_start(message_id="msg-001"))
print(emitter.emit_text_content("Hello, "))
print(emitter.emit_text_content("world!"))
print(emitter.emit_text_end())

In [ ]:
# Emit tool call events
print("=== Tool Call Events ===")
print(emitter.emit_text_start(message_id="msg-002"))  # Message containing tool call
print(emitter.emit_tool_call_start("search_web", tool_call_id="tc-001"))
print(emitter.emit_tool_call_args('{"query": "python"}'))
print(emitter.emit_tool_call_end())
print(emitter.emit_text_end())

## Part 4: Creating an AG-UI Server

Now let's create a FastAPI server that implements the AG-UI protocol.

In [3]:
# Import FastAPI for testing
from fastapi.testclient import TestClient

# Create server in echo mode (no agent)
app = create_agui_server(
    title="Demo AG-UI Server",
    description="Demonstration of AG-UI protocol"
)

# Create test client
client = TestClient(app)

# Test health endpoint
response = client.get("/health")
print(f"Health check: {response.json()}")

Health check: {'status': 'healthy', 'protocol': 'AG-UI'}


In [4]:
# Test streaming endpoint with echo mode
import json

request_data = {
    "thread_id": "thread-test-123",
    "run_id": "run-test-456",
    "messages": [
        {"role": "user", "content": "Hello, AG-UI!"}
    ]
}

# Make streaming request
with client.stream("POST", "/", json=request_data) as response:
    print("=== Streaming Response ===")
    for line in response.iter_lines():
        if line.startswith("data:"):
            event_data = json.loads(line[5:].strip())
            event_type = event_data.get("type")
            print(f"  {event_type}: {event_data}")

=== Streaming Response ===
  RUN_STARTED: {'type': 'RUN_STARTED', 'timestamp': 1767345887099, 'raw_event': None, 'thread_id': 'thread-test-123', 'run_id': 'run-test-456'}
  TEXT_MESSAGE_START: {'type': 'TEXT_MESSAGE_START', 'timestamp': 1767345887099, 'raw_event': None, 'message_id': 'msg-0ef8a4b07184', 'role': 'assistant'}
  TEXT_MESSAGE_CONTENT: {'type': 'TEXT_MESSAGE_CONTENT', 'timestamp': 1767345887099, 'raw_event': None, 'message_id': 'msg-0ef8a4b07184', 'delta': 'E'}
  TEXT_MESSAGE_CONTENT: {'type': 'TEXT_MESSAGE_CONTENT', 'timestamp': 1767345887099, 'raw_event': None, 'message_id': 'msg-0ef8a4b07184', 'delta': 'c'}
  TEXT_MESSAGE_CONTENT: {'type': 'TEXT_MESSAGE_CONTENT', 'timestamp': 1767345887099, 'raw_event': None, 'message_id': 'msg-0ef8a4b07184', 'delta': 'h'}
  TEXT_MESSAGE_CONTENT: {'type': 'TEXT_MESSAGE_CONTENT', 'timestamp': 1767345887099, 'raw_event': None, 'message_id': 'msg-0ef8a4b07184', 'delta': 'o'}
  TEXT_MESSAGE_CONTENT: {'type': 'TEXT_MESSAGE_CONTENT', 'timestam

## Part 5: Connecting Agent to AG-UI Server

Let's connect our ResearchAgent from Scenario 1 to stream responses.

In [5]:
# Import agent
from src.agents import ResearchAgent
from src.tools import search_web, calculate

# Create agent with tools
try:
    agent = ResearchAgent(name="streaming_agent")
    agent.set_tool_handlers({
        "search_web": search_web,
        "calculate": calculate,
    })
    
    # Create server with agent
    server = AGUIServer(agent=agent)
    agent_app = server.create_app()
    print("✅ Agent-connected server created!")
except Exception as e:
    print(f"⚠️ Could not create agent (Azure credentials may be missing): {e}")
    print("Using echo mode for demonstration...")
    agent_app = app  # Fall back to echo mode

✅ Agent-connected server created!


## Part 6: Understanding SSE Format

Server-Sent Events have a specific format:

```
event: <event-name>
data: <JSON payload>

```

The double newline separates events. Let's see how our events are formatted.

In [6]:
# Demonstrate SSE formatting
demo_emitter = AGUIEventEmitter(thread_id="t1", run_id="r1")

# Show raw SSE format
sse_event = demo_emitter.emit_run_started()
print("Raw SSE format:")
print(repr(sse_event))
print("\nFormatted:")
print(sse_event)

Raw SSE format:
'event: message\ndata: {"type":"RUN_STARTED","timestamp":1767345899682,"raw_event":null,"thread_id":"t1","run_id":"r1"}\n\n'

Formatted:
event: message
data: {"type":"RUN_STARTED","timestamp":1767345899682,"raw_event":null,"thread_id":"t1","run_id":"r1"}




## Part 7: Simulating Streaming Response

Let's simulate a complete streaming conversation.

In [7]:
async def simulate_streaming_response(user_message: str) -> None:
    """Simulate AG-UI streaming response."""
    emitter = AGUIEventEmitter(
        thread_id="thread-sim-001",
        run_id="run-sim-001"
    )
    
    # Simulate agent response
    response_text = f"I received your message: '{user_message}'. Let me help you with that."
    
    print("=== Simulated AG-UI Stream ===")
    
    # Run started
    print(emitter.emit_run_started())
    
    # Message start
    print(emitter.emit_text_start())
    
    # Stream tokens (word by word)
    words = response_text.split(" ")
    for i, word in enumerate(words):
        token = word if i == 0 else " " + word
        print(emitter.emit_text_content(token))
        await asyncio.sleep(0.05)  # Simulate streaming delay
    
    # Message end
    print(emitter.emit_text_end())
    
    # Run finished
    print(emitter.emit_run_finished())

# Run simulation
await simulate_streaming_response("What is the AG-UI protocol?")

=== Simulated AG-UI Stream ===
event: message
data: {"type":"RUN_STARTED","timestamp":1767345905494,"raw_event":null,"thread_id":"thread-sim-001","run_id":"run-sim-001"}


event: message
data: {"type":"TEXT_MESSAGE_START","timestamp":1767345905494,"raw_event":null,"message_id":"msg-caf24f18c884","role":"assistant"}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345905494,"raw_event":null,"message_id":"msg-caf24f18c884","delta":"I"}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345905548,"raw_event":null,"message_id":"msg-caf24f18c884","delta":" received"}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345905606,"raw_event":null,"message_id":"msg-caf24f18c884","delta":" your"}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345905663,"raw_event":null,"message_id":"msg-caf24f18c884","delta":" message:"}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345905714,"raw_event":nul

## Part 8: Tool Call Streaming

When an agent calls a tool, we stream tool execution events.

In [8]:
async def simulate_tool_call_stream() -> None:
    """Simulate AG-UI stream with tool call."""
    emitter = AGUIEventEmitter(
        thread_id="thread-tool-001",
        run_id="run-tool-001"
    )
    
    print("=== Simulated Tool Call Stream ===")
    
    # Run started
    print(emitter.emit_run_started())
    
    # Assistant decides to use tool
    print(emitter.emit_text_start())
    print(emitter.emit_text_content("Let me search for that information..."))
    print(emitter.emit_text_end())
    
    # Tool call begins
    print(emitter.emit_text_start())  # New message for tool call
    print(emitter.emit_tool_call_start("search_web"))
    
    # Stream tool arguments
    args_json = '{"query": "AG-UI protocol", "max_results": 5}'
    for char in args_json:
        print(emitter.emit_tool_call_args(char))
        await asyncio.sleep(0.01)
    
    # Tool call ends
    print(emitter.emit_tool_call_end())
    print(emitter.emit_text_end())
    
    # Response after tool execution
    print(emitter.emit_text_start())
    print(emitter.emit_text_content("Based on my search, AG-UI is a protocol for..."))
    print(emitter.emit_text_end())
    
    # Run finished
    print(emitter.emit_run_finished())

await simulate_tool_call_stream()

=== Simulated Tool Call Stream ===
event: message
data: {"type":"RUN_STARTED","timestamp":1767345913070,"raw_event":null,"thread_id":"thread-tool-001","run_id":"run-tool-001"}


event: message
data: {"type":"TEXT_MESSAGE_START","timestamp":1767345913070,"raw_event":null,"message_id":"msg-f519bd2bda9e","role":"assistant"}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345913070,"raw_event":null,"message_id":"msg-f519bd2bda9e","delta":"Let me search for that information..."}


event: message
data: {"type":"TEXT_MESSAGE_END","timestamp":1767345913070,"raw_event":null,"message_id":"msg-f519bd2bda9e"}


event: message
data: {"type":"TEXT_MESSAGE_START","timestamp":1767345913070,"raw_event":null,"message_id":"msg-e9c89ff9294f","role":"assistant"}


event: message
data: {"type":"TOOL_CALL_START","timestamp":1767345913071,"raw_event":null,"tool_call_id":"tc-611e6cd48baa","tool_call_name":"search_web","parent_message_id":"msg-e9c89ff9294f"}


event: message
data: {"type":

## Part 9: Error Handling

AG-UI includes error events for graceful error handling.

In [9]:
# Demonstrate error handling
emitter = AGUIEventEmitter(thread_id="t-err", run_id="r-err")

print("=== Error Handling ===")
print(emitter.emit_run_started())
print(emitter.emit_run_error(
    message="Connection timeout while processing request",
    code="TIMEOUT_ERROR"
))

=== Error Handling ===
event: message
data: {"type":"RUN_STARTED","timestamp":1767345928761,"raw_event":null,"thread_id":"t-err","run_id":"r-err"}


event: message
data: {"type":"RUN_ERROR","timestamp":1767345928761,"raw_event":null,"message":"Connection timeout while processing request","code":"TIMEOUT_ERROR"}




In [10]:
# Test error conditions
from src.common.exceptions import AGUIError

error_emitter = AGUIEventEmitter(thread_id="t1", run_id="r1")

# Try to emit content without starting message
try:
    error_emitter.emit_text_content("test")
except AGUIError as e:
    print(f"✅ Caught expected error: {e}")

# Try to emit empty content
error_emitter.emit_text_start()
try:
    error_emitter.emit_text_content("")
except AGUIError as e:
    print(f"✅ Caught expected error: {e}")

✅ Caught expected error: No active message (delta=test)
✅ Caught expected error: Delta cannot be empty


## Part 10: Edge Case - Connection Drop

In real applications, connections can drop. Here's how to handle reconnection.

In [11]:
from dataclasses import dataclass, field
from typing import Optional

@dataclass
class ReconnectionState:
    """Track state for reconnection handling."""
    last_event_id: int = 0
    last_message_id: Optional[str] = None
    partial_content: str = ""
    reconnect_count: int = 0
    max_reconnects: int = 3

async def simulate_reconnection() -> None:
    """Demonstrate reconnection handling."""
    state = ReconnectionState()
    
    print("=== Reconnection Scenario ===")
    
    # Simulate initial connection
    print("\n1. Initial connection established")
    emitter = AGUIEventEmitter(thread_id="t-reconnect", run_id="r-reconnect")
    print(emitter.emit_run_started())
    print(emitter.emit_text_start())
    
    # Stream some content
    content = "This is a long response that will be interrupted..."
    for word in content.split()[:5]:  # Only first 5 words
        state.partial_content += word + " "
        print(emitter.emit_text_content(word + " "))
        state.last_event_id += 1
    
    # Simulate disconnect
    print("\n⚠️ CONNECTION DROPPED!")
    state.reconnect_count += 1
    
    # Client reconnects
    print(f"\n2. Reconnecting... (attempt {state.reconnect_count}/{state.max_reconnects})")
    
    # Send state snapshot for recovery
    recovery_state = {
        "partial_content": state.partial_content.strip(),
        "last_event_id": state.last_event_id,
        "status": "resuming"
    }
    print(emitter.emit_state_snapshot(recovery_state))
    
    # Continue streaming
    remaining = content.split()[5:]  # Rest of words
    for word in remaining:
        print(emitter.emit_text_content(word + " "))
    
    print(emitter.emit_text_end())
    print(emitter.emit_run_finished())
    print("\n✅ Stream completed successfully after reconnection")

await simulate_reconnection()

=== Reconnection Scenario ===

1. Initial connection established
event: message
data: {"type":"RUN_STARTED","timestamp":1767345963694,"raw_event":null,"thread_id":"t-reconnect","run_id":"r-reconnect"}


event: message
data: {"type":"TEXT_MESSAGE_START","timestamp":1767345963694,"raw_event":null,"message_id":"msg-4d92dc636bce","role":"assistant"}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345963694,"raw_event":null,"message_id":"msg-4d92dc636bce","delta":"This "}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345963694,"raw_event":null,"message_id":"msg-4d92dc636bce","delta":"is "}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345963694,"raw_event":null,"message_id":"msg-4d92dc636bce","delta":"a "}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345963694,"raw_event":null,"message_id":"msg-4d92dc636bce","delta":"long "}


event: message
data: {"type":"TEXT_MESSAGE_CONTENT","timestamp":1767345

## Part 11: State Management

AG-UI supports state snapshots and deltas for complex applications.

In [12]:
# Demonstrate state management
state_emitter = AGUIEventEmitter(thread_id="t-state", run_id="r-state")

print("=== State Management ===")

# Initial state snapshot
initial_state = {
    "conversation_id": "conv-001",
    "message_count": 0,
    "tools_called": [],
    "tokens_used": 0
}
print(state_emitter.emit_state_snapshot(initial_state))

# State deltas as things change
print(state_emitter.emit_state_delta({"message_count": 1}))
print(state_emitter.emit_state_delta({"tools_called": ["search_web"]}))
print(state_emitter.emit_state_delta({"tokens_used": 150}))

=== State Management ===
event: message
data: {"type":"STATE_SNAPSHOT","timestamp":1767345968240,"raw_event":null,"state":{"conversation_id":"conv-001","message_count":0,"tools_called":[],"tokens_used":0}}


event: message
data: {"type":"STATE_DELTA","timestamp":1767345968242,"raw_event":null,"delta":{"message_count":1}}


event: message
data: {"type":"STATE_DELTA","timestamp":1767345968243,"raw_event":null,"delta":{"tools_called":["search_web"]}}


event: message
data: {"type":"STATE_DELTA","timestamp":1767345968243,"raw_event":null,"delta":{"tokens_used":150}}




## Part 12: Hands-On Exercise

### Exercise: Customize UI Component Rendering

Create a custom event processor that formats events for a terminal UI.

In [ ]:
# Exercise: Complete this event processor

class TerminalUIProcessor:
    """Process AG-UI events for terminal display."""
    
    def __init__(self):
        self.current_content = ""
    
    def process_event(self, event_dict: dict) -> str:
        """Process an event and return formatted output."""
        event_type = event_dict.get("type")
        
        if event_type == "RUN_STARTED":
            return "🚀 Starting..."
        
        elif event_type == "TEXT_MESSAGE_START":
            self.current_content = ""
            return "\n🤖 Assistant: "
        
        elif event_type == "TEXT_MESSAGE_CONTENT":
            delta = event_dict.get("delta", "")
            self.current_content += delta
            return delta  # Return just the new content
        
        elif event_type == "TEXT_MESSAGE_END":
            return "\n"
        
        elif event_type == "TOOL_CALL_START":
            tool_name = event_dict.get("tool_call_name", "unknown")
            return f"\n🔧 Calling tool: {tool_name}\n"
        
        elif event_type == "RUN_FINISHED":
            return "\n✅ Complete!"
        
        elif event_type == "RUN_ERROR":
            message = event_dict.get("message", "Unknown error")
            return f"\n❌ Error: {message}"
        
        # TODO: Add handling for TOOL_CALL_ARGS, TOOL_CALL_END, STATE_SNAPSHOT
        
        return ""

# Test the processor
processor = TerminalUIProcessor()

test_events = [
    {"type": "RUN_STARTED", "thread_id": "t1", "run_id": "r1"},
    {"type": "TEXT_MESSAGE_START", "message_id": "m1", "role": "assistant"},
    {"type": "TEXT_MESSAGE_CONTENT", "message_id": "m1", "delta": "Hello"},
    {"type": "TEXT_MESSAGE_CONTENT", "message_id": "m1", "delta": " world!"},
    {"type": "TEXT_MESSAGE_END", "message_id": "m1"},
    {"type": "RUN_FINISHED", "thread_id": "t1", "run_id": "r1"},
]

print("=== Terminal UI Output ===")
for event in test_events:
    output = processor.process_event(event)
    print(output, end="", flush=True)

## Summary

In this scenario, you learned:

1. **AG-UI Protocol**: Event-based protocol for agent-UI communication
2. **Event Types**: Lifecycle, message, tool, and state events
3. **SSE Format**: Server-Sent Events for streaming
4. **Event Emitter**: Creating and formatting AG-UI events
5. **FastAPI Server**: Building AG-UI endpoints
6. **Error Handling**: Graceful error events
7. **Reconnection**: State recovery after disconnects

## Next Steps

- **Scenario 3**: A2A Protocol for agent-to-agent communication
- Build a full web UI using these events
- Implement progress indicators for long operations

## Resources

- [AG-UI Protocol Spec](specs/001-agentic-patterns-workshop/contracts/agui-events.md)
- [Server-Sent Events (MDN)](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events)
- [FastAPI Streaming](https://fastapi.tiangolo.com/advanced/custom-response/#streamingresponse)